In [1]:
__author__ = 'Anderson Carlos Ferreira da Silva'

import sys
import logging
from operator import attrgetter
from skmultiflow.core.utils.utils import *
from skmultiflow.classification.base import BaseClassifier
from skmultiflow.classification.trees.hoeffding_tree import HoeffdingTree
from random import randint

FEATURES_M = ''
FEATURES_SQRT = 'sqrt'
FEATURES_SQRT_INV = 'sqrt_inv'
FEATURES_PERCENT = 'percent'

class ADFHoeffdingTree(HoeffdingTree):
            
    class RandomLearningNode(HoeffdingTree.ActiveLearningNode):                    
        """Random learning node.
        Parameters
        ----------
        initial_class_observations: dict (class_value, weight) or None
            Initial class observations
        """
        def __init__(self, initial_class_observations, nb_attributes):
            super().__init__(initial_class_observations)
            self.nb_attributes = nb_attributes;
            self.list_attributes = None
            
        def learn_from_instance(self, X, y, weight, ht):
            """Update the node with the provided instance.
            Parameters
            ----------
            X: numpy.ndarray of length equal to the number of features.
                Instance attributes for updating the node.
            y: int
                Instance class.
            weight: float
                Instance weight.
            ht: HoeffdingTree
                Hoeffding Tree to update.
            """            
            self._observed_class_distribution[y] += weight                            
            if not self.list_attributes:
                self.list_attributes = [None] * self.nb_attributes
                for j in range(self.nb_attributes):    
                    is_unique = False
                    while is_unique == False:
                        self.list_attributes[j] = randint(0, self.nb_attributes - 1)
                        is_unique = True
                        for i in range(j):
                            if self.list_attributes[j] == self.list_attributes[i]:
                                is_unique = False
                                break
            
            for j in range(self.nb_attributes):
                i = self.list_attributes[j]
                obs = self._attribute_observers[i]
                if obs is None:
                    if i in ht.nominal_attributes:
                        obs = NominalAttributeClassObserver()
                    else:
                        obs = GaussianNumericAttributeClassObserver()
                    self._attribute_observers[i] = obs
            obs.observe_attribute_class(X[i], int(y), weight)
            
    class LearningNodeNB(RandomLearningNode):

        def __init__(self, initial_class_observations, nb_attributes):
            super().__init__(initial_class_observations, nb_attributes)            
            
        def get_class_votes(self, X, ht):
            """Get the votes per class for a given instance.
            Parameters
            ----------
            X: numpy.ndarray of length equal to the number of features.
                Instance attributes.
            ht: HoeffdingTree
                Hoeffding Tree.
            Returns
            -------
            dict (class_value, weight)
                Class votes for the given instance.
            """
            if self.get_weight_seen() >= ht.nb_threshold:
                return do_naive_bayes_prediction(X, self._observed_class_distribution, self._attribute_observers)
            else:
                return super().get_class_votes(X, ht)

    class LearningNodeNBAdaptive(LearningNodeNB):
        """Learning node that uses Adaptive Naive Bayes models.
        Parameters
        ----------
        initial_class_observations: dict (class_value, weight) or None
            Initial class observations
        """
        def __init__(self, initial_class_observations, nb_attributes):
            """LearningNodeNBAdaptive class constructor. """
            super().__init__(initial_class_observations, nb_attributes)
            self._mc_correct_weight = 0.0
            self._nb_correct_weight = 0.0

        def learn_from_instance(self, X, y, weight, ht):
            """Update the node with the provided instance.
            Parameters
            ----------
            X: numpy.ndarray of length equal to the number of features.
                Instance attributes for updating the node.
            y: int
                Instance class.
            weight: float
                The instance's weight.
            ht: HoeffdingTree
                The Hoeffding Tree to update.
            """
            if self._observed_class_distribution == {}:
                # All classes equal, default to class 0
                if 0 == y:
                    self._mc_correct_weight += weight
            elif max(self._observed_class_distribution, key=self._observed_class_distribution.get) == y:
                self._mc_correct_weight += weight
            nb_prediction = do_naive_bayes_prediction(X, self._observed_class_distribution, self._attribute_observers)
            if max(nb_prediction, key=nb_prediction.get) == y:
                self._nb_correct_weight += weight
            super().learn_from_instance(X, y, weight, ht)

        def get_class_votes(self, X, ht):
            """Get the votes per class for a given instance.
            Parameters
            ----------
            X: numpy.ndarray of length equal to the number of features.
                Instance attributes.
            ht: HoeffdingTree
                Hoeffding Tree.
            Returns
            -------
            dict (class_value, weight)
                Class votes for the given instance.
            """
            if self._mc_correct_weight > self._nb_correct_weight:
                return self._observed_class_distribution
            return do_naive_bayes_prediction(X, self._observed_class_distribution, self._attribute_observers)
        
    def __init__(self, max_byte_size = 33554432, memory_estimate_period = 1000000, grace_period = 200,
                     split_criterion = 'info_gain', split_confidence = 0.0000001, tie_threshold = 0.05, 
                     binary_split = False, stop_mem_management = False, remove_poor_atts = False, no_preprune = False, 
                     leaf_prediction = 'mc', nb_threshold = 0, nominal_attributes = None, nb_attributes = 2):                
        """ADFHoeffdingTree class constructor."""
        super().__init__(max_byte_size, memory_estimate_period, grace_period, split_criterion, split_confidence,
                        tie_threshold, binary_split, stop_mem_management, remove_poor_atts, no_preprune,
                        leaf_prediction, nb_threshold, nominal_attributes)
        self.nb_attributes = nb_attributes
        self.remove_poor_attributes_option = None        

    def newLearningNode(self, initial_class_observations):        
        if self._leaf_prediction == MAJORITY_CLASS:
            return RandomLearningNode(self, initialClassObservations, self.nb_attributes)            
        elif self._leaf_prediction == NAIVE_BAYES:
            return LearningNodeNB(self, initialClassObservations, self.nb_attributes)            
        else: #NAIVE_BAYES_ADAPTIVE
            return LearningNodeNBAdaptative(self, initialClassObservations, self.nb_attributes)
            
    def isRandomizable():
        return True;
    

# Tests for ADFHoeffdingtree.

## For the following part we test the ADFHoeffdingtree algorithm according to several parameters:
    -If the stream is created with a csv file or with the waveformgenerator()
    
    -We test for 3 datasets: covtype, movingSquares and sea_stream.
    
        -covtype: The forest covertype data set represents forest cover type for 30 x30 meter cells obtained from the US Forest Service Region 2 Resource InformationSystem (RIS) data. Each class corresponds to a diﬀerent cover type. This dataset contains 581,012 instances, 54 attributes (10 numeric and 44 binary) and 7imbalanced class labels. 
       
       -movingsquares:
        
       -sea_stream: The SEA generator produces data streams with three continuousattributes (f1, f2, f3). The range of values that each attribute can assume is be-tween 0 and 10. Only the ﬁrst two attributes (f1, f2) are relevant, i.e., f3doesnot inﬂuence the class value determination. New instances are obtained throughrandomly setting a point in a two dimensional space, such that these dimensionscorresponds to f1and f2. This two dimensional space is split into four blocks,each of which corresponds to one of four diﬀerent functions. In each block a pointbelongs to class 1 if f1+f2≤θand to class 0 otherwise. The threshold θusedto split instances between class 0 and 1 assumes values 8 (block 1), 9 (block 2), 7(block 3) and 9.5 (block 4). It is possible to add noise to class values, being thedefault value 10%, and to balance the number of instances of each class. SEAgsimulates 3 gradual drifts, while SEAasimulates 3 abrupt drifts 

Adaptive random forests for evolving data stream classification (PDF Download Available). Available from: https://www.researchgate.net/publication/317579226_Adaptive_random_forests_for_evolving_data_stream_classification [accessed Feb 11 2018].
       
    -We use 2 evaluators: Prequential and Holdout. Hold-out is more accurate, but needs data for testing.
    
        -Prequential:The error of a model is computed from the sequence of examples. 
        For each example in the stream, the actual model makes a prediction, and then uses it to update the model.
        
        -Holdout:Apply the current decision model to the test set,at regular time intervals 
        The loss estimated in the hold out is an unbiased estimator
        
    -Finally we compare the model with a naive bayes model, BernoulliNB, with the accuracy and the Kappa statistic.
        
        -Accuracy:
        
        -Kappa statistic: k=1 the classifier is always correct. 
                          k=0 the predictions coincide with the correct ones as often as those of the chance classifier
        
        
ADFHoeffdingtree with wave form generator stream

In [2]:
from skmultiflow.data.generators.waveform_generator import WaveformGenerator
from skmultiflow.classification.trees.hoeffding_tree import HoeffdingTree
from skmultiflow.evaluation.evaluate_prequential import EvaluatePrequential

# 1. Create a stream
stream = WaveformGenerator()
stream.prepare_for_use()

# 2. Instantiate the classifier
adfht = ADFHoeffdingTree()

# 3. Setup the evaluator
eval = EvaluatePrequential(show_plot=True, pretrain_size=100, max_instances=10000)

# 4. Run evaluation
eval.eval(stream=stream, classifier=adfht)

skmultiflow.classification.trees.hoeffding_tree - INFO - No Nominal attributes have been defined, will consider all attributes as numerical
root - INFO - Prequential Evaluation
root - INFO - Generating 3 targets.
root - INFO - Pretraining on 100 samples.
root - INFO - Evaluating...


root - INFO - 5.0%


root - INFO - 10.0%


root - INFO - 15.0%


root - INFO - 20.0%


...

root - INFO - 80.0%


root - INFO - 85.0%


root - INFO - 90.0%


root - INFO - 95.0%


root - INFO - 100.0%
root - INFO - 
Evaluation time: 8.262 s
root - INFO - Total instances: 10000
root - INFO - Classifier 0 - Global accuracy: 0.377
root - INFO - Classifier 0 - Global kappa: 0.063


1. Error estimation: Hold-out or Prequential
2. Evaluation performance measures: Accuracy or κ-statistic 
3. Statistical significance validation: MacNemar or Nemenyi test 
4. Resources needed: time and memory or RAM-Hours

Eval Pre and Holdout with datasets.csv for ADFHoeddfing tree


In [8]:
from skmultiflow.options.file_option import FileOption
from skmultiflow.data.file_stream import FileStream
from skmultiflow.classification.trees.hoeffding_tree import HoeffdingTree
from skmultiflow.evaluation.evaluate_prequential import EvaluatePrequential
from skmultiflow.evaluation.evaluate_holdout import EvaluateHoldout


# 1. Create a stream
options = FileOption(option_value="../datasets/covtype.csv", file_extension="CSV")
#options = FileOption(option_value="../datasets/movingSquares.csv", file_extension="CSV")
#options = FileOption(option_value="../datasets/sea_stream.csv", file_extension="CSV")

stream = FileStream(options)

stream.prepare_for_use()

# 2. Instantiate the classifier
adfht = ADFHoeffdingTree()

# 3. Setup the evaluator
eval = EvaluatePrequential(pretrain_size=1000, max_instances=100000, batch_size=1, max_time=1000, output_file='resultsPrequential.csv', task_type='classification', show_plot=False, plot_options=['kappa', 'kappa_t', 'performance'])
#eval = EvaluateHoldout(pretrain_size=200, max_instances=100000, batch_size=1, max_time=1000, output_file='resultsHoldout.csv', task_type='classification', show_plot=False, plot_options=['kappa', 'performance'], test_size=5000, dynamic_test_set=True)

# 4. Run evaluation
eval.eval(stream=stream, classifier=adfht)

skmultiflow.classification.trees.hoeffding_tree - INFO - No Nominal attributes have been defined, will consider all attributes as numerical
root - INFO - Prequential Evaluation
root - INFO - Generating 7 targets.
root - INFO - Pretraining on 1000 samples.
root - INFO - Evaluating...
root - INFO - 5.0%
root - INFO - 10.0%
root - INFO - 15.0%
root - INFO - 20.0%
root - INFO - 25.0%
root - INFO - 30.0%
root - INFO - 35.0%
root - INFO - 40.0%
root - INFO - 45.0%
root - INFO - 50.0%
root - INFO - 55.0%
root - INFO - 60.0%
root - INFO - 65.0%
root - INFO - 70.0%
root - INFO - 75.0%
root - INFO - 80.0%
root - INFO - 85.0%
root - INFO - 90.0%
root - INFO - 95.0%
root - INFO - 100.0%
root - INFO - 
Evaluation time: 177.301 s
root - INFO - Total instances: 100000
root - INFO - Classifier 0 - Global accuracy: 0.727
root - INFO - Classifier 0 - Global kappa: 0.418
root - INFO - Classifier 0 - Global kappa T: -2.03


compare two classifiers with holdout

In [4]:
# The second example will demonstrate how to compare two classifiers with
# the EvaluateHoldout
from skmultiflow.evaluation.evaluate_holdout import EvaluateHoldout

from sklearn.naive_bayes import BernoulliNB

options = FileOption(option_value="../datasets/covtype.csv", file_extension="CSV")
#options = FileOption(option_value="../datasets/movingSquares.csv", file_extension="CSV")
#options = FileOption(option_value="../datasets/sea_stream.csv", file_extension="CSV")
stream = FileStream(options)

stream.prepare_for_use()

clf_one = BernoulliNB()
clf_two = ADFHoeffdingTree()
classifier = [clf_one, clf_two]

eval = EvaluateHoldout(pretrain_size=200, test_size=5000, dynamic_test_set=True, max_instances=100000, batch_size=1,  max_time=1000, output_file='comparison_Bernoulli_ADFH_Holdout.csv', task_type='classification', show_plot=False, plot_options=['kappa', 'performance'])
eval.eval(stream=stream, classifier=classifier)

skmultiflow.classification.trees.hoeffding_tree - INFO - No Nominal attributes have been defined, will consider all attributes as numerical
root - INFO - Holdout Evaluation
root - INFO - Generating 7 targets.
root - INFO - Pretraining on 200 samples.
/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
root - INFO - Evaluating...
root - INFO - 5.0%
root - INFO - 10.0%
root - INFO - Separating 5000 dynamic holdout samples.
root - INFO - Testing model on 5000 samples.
root - INFO - 15.0%
root - INFO - 20.0%
root - INFO - Separating 5000 dynamic holdout samples.
root - INFO - Testing model on 5000 samples.
root - INFO - 25.0%
root - INFO - 30.0%
root - INFO - Separating 5000 dynamic holdout samples.
root - INFO - Testing model on 5000 samples.
root - INFO - 35.0%
root - INFO - 40.0%
root - INFO - Separating 5000 dynamic holdout samples.
root - INFO - Testing mod

[BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),

compare two classifiers with prequential

In [5]:
# The second example will demonstrate how to compare two classifiers with
# the EvaluatePrequential
from skmultiflow.core.pipeline import Pipeline
from skmultiflow.data.file_stream import FileStream
from skmultiflow.options.file_option import FileOption
from skmultiflow.evaluation.evaluate_prequential import EvaluatePrequential
from sklearn.naive_bayes import BernoulliNB


# Setup the File Stream
opt = FileOption("FILE", "OPT_NAME", "../datasets/covtype.csv", "CSV", False)
#opt = FileOption("FILE", "OPT_NAME", "../datasets/movingSquares.csv", "CSV", False)
#opt = FileOption("FILE", "OPT_NAME", "../datasets/sea_stream.csv", "CSV", False)

stream = FileStream(opt, -1, 1)
stream.prepare_for_use()

# Setup the classifiers
clf_one = BernoulliNB()
clf_two = ADFHoeffdingTree()

# Setup the pipeline for clf_one
pipe = Pipeline([('Classifier', clf_one)])

# Create the list to hold both classifiers
classifier = [pipe, clf_two]

# Setup the evaluator
eval = EvaluatePrequential(pretrain_size=200, max_instances=100000, batch_size=1, max_time=1000, output_file='comparison_Bernoulli_ADFH_Preq.csv', task_type='classification', show_plot=False, plot_options=['kappa', 'kappa_t', 'performance'])

# Evaluate
eval.eval(stream=stream, classifier=classifier)

skmultiflow.classification.trees.hoeffding_tree - INFO - No Nominal attributes have been defined, will consider all attributes as numerical
root - INFO - Prequential Evaluation
root - INFO - Generating 7 targets.
root - INFO - Pretraining on 200 samples.
/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -
root - INFO - Evaluating...
root - INFO - 5.0%
root - INFO - 10.0%
root - INFO - 15.0%
root - INFO - 20.0%
root - INFO - 25.0%
root - INFO - 30.0%
root - INFO - 35.0%
root - INFO - 40.0%
root - INFO - 45.0%
root - INFO - 50.0%
root - INFO - 55.0%
root - INFO - 60.0%
root - INFO - 65.0%
root - INFO - 70.0%
root - INFO - 75.0%
root - INFO - 80.0%
root - INFO - 85.0%
root - INFO - 90.0%
root - INFO - 95.0%
root - INFO - 100.0%
root - INFO - 
Evaluation time: 271.276 s
root - INFO - Total instances: 100000
root - INFO - Classifier 0 - Global accuracy: 0.741
roo

In [6]:
# Verification of the same result for ADFHoeffdingTree
from skmultiflow.core.pipeline import Pipeline
from skmultiflow.data.file_stream import FileStream
from skmultiflow.options.file_option import FileOption
from skmultiflow.evaluation.evaluate_prequential import EvaluatePrequential
from skmultiflow.classification.trees.hoeffding_tree import HoeffdingTree


# Setup the File Stream
opt = FileOption("FILE", "OPT_NAME", "../datasets/covtype.csv", "CSV", False)
#opt = FileOption("FILE", "OPT_NAME", "../datasets/movingSquares.csv", "CSV", False)
#opt = FileOption("FILE", "OPT_NAME", "../datasets/sea_stream.csv", "CSV", False)
stream = FileStream(opt, -1, 1)
stream.prepare_for_use()

# Setup the classifiers
clf_one = HoeffdingTree()
clf_two = ADFHoeffdingTree()

# Setup the pipeline for clf_one
pipe = Pipeline([('Classifier', clf_one)])

# Create the list to hold both classifiers
classifier = [pipe, clf_two]

# Setup the evaluator
eval = EvaluatePrequential(pretrain_size=200, max_instances=100000, batch_size=1, max_time=1000, output_file='comparison_Hoeffding_ADFH_Preq.csv', task_type='classification', plot_options=['kappa', 'kappa_t', 'performance'])

# Evaluate
eval.eval(stream=stream, classifier=classifier)

skmultiflow.classification.trees.hoeffding_tree - INFO - No Nominal attributes have been defined, will consider all attributes as numerical
skmultiflow.classification.trees.hoeffding_tree - INFO - No Nominal attributes have been defined, will consider all attributes as numerical
root - INFO - Prequential Evaluation
root - INFO - Generating 7 targets.
root - INFO - Pretraining on 200 samples.
root - INFO - Evaluating...
root - INFO - 5.0%
root - INFO - 10.0%
root - INFO - 15.0%
root - INFO - 20.0%
root - INFO - 25.0%
root - INFO - 30.0%
root - INFO - 35.0%
root - INFO - 40.0%
root - INFO - 45.0%
root - INFO - 50.0%
root - INFO - 55.0%
root - INFO - 60.0%
root - INFO - 65.0%
root - INFO - 70.0%
root - INFO - 75.0%
root - INFO - 80.0%
root - INFO - 85.0%
root - INFO - 90.0%
root - INFO - 95.0%
root - INFO - 100.0%
root - INFO - 
Evaluation time: 322.487 s
root - INFO - Total instances: 100000
root - INFO - Classifier 0 - Global accuracy: 0.727
root - INFO - Classifier 0 - Global kappa: 0.